In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Strawberry___healthy', 'Strawberry___Leaf_scorch']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(10820, 128, 128, 3)

In [8]:
y.shape

(10820, 6)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(8656, 128, 128, 3)

In [11]:
X_test.shape

(2164, 128, 128, 3)

In [12]:
y_train.shape

(8656, 6)

In [13]:
y_test.shape

(2164, 6)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

6

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 11:06:36.490077: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


271/271 [==============================] - 70s 257ms/step - loss: 0.4800 - accuracy: 0.8265 - val_loss: 0.4226 - val_accuracy: 0.8420
Epoch 2/10
271/271 [==============================] - 64s 236ms/step - loss: 0.1723 - accuracy: 0.9385 - val_loss: 0.3633 - val_accuracy: 0.8891
Epoch 3/10
271/271 [==============================] - 69s 256ms/step - loss: 0.1018 - accuracy: 0.9649 - val_loss: 0.1531 - val_accuracy: 0.9459
Epoch 4/10
271/271 [==============================] - 67s 248ms/step - loss: 0.0817 - accuracy: 0.9702 - val_loss: 0.1478 - val_accuracy: 0.9579
Epoch 5/10
271/271 [==============================] - 69s 255ms/step - loss: 0.0665 - accuracy: 0.9770 - val_loss: 0.3307 - val_accuracy: 0.8803
Epoch 6/10
271/271 [==============================] - 69s 254ms/step - loss: 0.0490 - accuracy: 0.9829 - val_loss: 0.0737 - val_accuracy: 0.9764
Epoch 7/10
271/271 [==============================] - 67s 248ms/step - loss: 0.0391 - accuracy: 0.9867 - val_loss: 0.3546 - val_accuracy: 0.8

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 94.69%


In [19]:
cnn_scores

[0.15891137719154358, 0.9468576908111572]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

68/68 [==============================] - 6s 83ms/step
Accuracy: 94.69%
Precision: 95.00%
Recall: 94.69%
F1 score: 94.49%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-anggur+strawberry.csv', index=False)

+------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                 | True Labels                                | Predicted Labels                           |
|------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|    0 | 85726470-e247-4d82-89d1-f6adad570519___RS_HL 2150_90deg.JPG              | Strawberry___healthy                       | Strawberry___healthy                       |
|    1 | 3c7cac7d-a1f3-4534-854c-7ae924ac4a88___FAM_B.Msls 1043.JPG               | Grape___Esca_(Black_Measles)               | Grape___Esca_(Black_Measles)               |
|    2 | f6bca207-2928-4753-b867-ec5f25361334___Mt.N.V_HL 9113_180deg.JPG         | Grape___healthy                            | G

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-anggur+strawberry.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                 | True Labels                                | Predicted Labels                           |
|------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|   29 | be4475e2-a944-42de-97f2-5e88d7b3de56___FAM_L.Blight 1528.JPG             | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) | Grape___Black_rot                          |
|   55 | 1e126702-2613-4265-b2df-03648d947627___Mt.N.V_HL 9108_180deg.JPG         | Grape___healthy                            | Strawberry___healthy                       |
|   56 | 9d4aab70-faaf-4bef-9b6b-b214ff80f687___Mt.N.V_HL 6154_90deg.JPG          | Grape___healthy            

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

271/271 [==============================] - 37s 137ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 223.62819290161133 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

68/68 [==============================] - 8s 122ms/step
Random Forest Classifier Accuracy: 99.40%
Random Forest Classifier Precision: 0.99
Random Forest Classifier Recall: 0.99
Random Forest Classifier F1 Score: 0.99


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-anggur+strawberry.csv', index=False)

+------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                 | True Labels                                | Predicted Labels                           |
|------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|    0 | 85726470-e247-4d82-89d1-f6adad570519___RS_HL 2150_90deg.JPG              | Strawberry___healthy                       | Strawberry___healthy                       |
|    1 | 3c7cac7d-a1f3-4534-854c-7ae924ac4a88___FAM_B.Msls 1043.JPG               | Grape___Esca_(Black_Measles)               | Grape___Esca_(Black_Measles)               |
|    2 | f6bca207-2928-4753-b867-ec5f25361334___Mt.N.V_HL 9113_180deg.JPG         | Grape___healthy                            | G

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-anggur+strawberry.csv', index=False)

False Predictions:
+------+---------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                            | True Labels                                | Predicted Labels                           |
|------+---------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|   29 | be4475e2-a944-42de-97f2-5e88d7b3de56___FAM_L.Blight 1528.JPG        | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) | Grape___Black_rot                          |
|  326 | 9a668952-936a-4d6e-8cda-31f4dcd95a6b___FAM_B.Rot 0458_flipLR.JPG    | Grape___Black_rot                          | Grape___healthy                            |
|  346 | ace83c97-ae73-40be-9cd8-293ea67f4f5a___FAM_B.Msls 1206.JPG          | Grape___Esca_(Black_Measles)               | Grape___Blac